## Data generation notebook
used for inpection of teh synthetic dataset and tuning

In [1]:

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #for plotting the 3-D plot.
from src.data_generator.labels_generator import Label_generator
from src.data_generator.data_gen_utils import *
from src.motion_refiner_4D import Motion_refiner
from src.functions import *
from src.config import *

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# warnings.filterwarnings("ignore", category=FutureWarning) 


%load_ext autoreload
%autoreload 2

/home/arthur/.conda/envs/py38_cu11_2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mr = Motion_refiner(traj_n = traj_n, load_models = True, clip_only=False, locality_factor=True, poses_on_features=True, load_precomp_emb=True)
obj_lib_file= image_dataset_folder+"imagenet1000_clsidx_to_labels.txt"
dataset_name = "test"


loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
loading precomputed CLIP text embbedings... done
loading precomputed CLIP img embbedings... done
DEVICE:  cuda


## Cartesian, distance and speed commands

In [3]:

dg = data_generator({'dist':1, 'cartesian':1, 'speed':1}, obj_lib_file= obj_lib_file, images_base_path=image_dataset_folder)
data = dg.generate(2,4,N=[20,100],n_int=[5,15])
n_map = 2
labels_per_map = 4
data = dg.generate(n_map,labels_per_map,N=[50,100],n_int=[3,15])
print(len(data))

## ------- processed data -------
# X,Y = mr.prepare_data(data,deltas=False)
# print("X: ",X.shape)
# print("Y: ",Y.shape)
# print("DONE computing embeddings")
# print("saving data...")
# ------- save pre processed data -------
# mr.save_XY(X, Y, x_name="X"+dataset_name,y_name="Y"+dataset_name)
# mr.save_data(data,data_name="data"+dataset_name)
# print("DONE")
%matplotlib qt

data_sample = random.choices(data,k=3)
show_data4D(data_sample)#,image_loader=mr.image_loader)

100%|██████████| 2/2 [00:01<00:00,  1.84it/s]


8


## Force commands

In [4]:
dg = data_generator({'force':1,'cartesian force':1}, obj_lib_file= obj_lib_file, images_base_path=image_dataset_folder)
data = dg.generate(1,3,N=[20,100],n_int=[5,15])
n_map = 10
labels_per_map = 1
data = dg.generate(n_map,labels_per_map,N=[50,100],n_int=[3,15],output_forces=True)
print(len(data))

# ------- processed data -------
# X,Y = mr.prepare_data(data,deltas=False,output_forces=True, change_img_base=['/home/arthur/image_dataset/','/home/arthur/data/image_dataset//'])
# print("X: ",X.shape)
# print("Y: ",Y.shape)
# print("DONE computing embeddings")
# print("saving data...")
# # ------- save pre processed data -------
# mr.save_XY(X, Y, x_name="X"+dataset_name,y_name="Y"+dataset_name)
# mr.save_data(data,data_name="data"+dataset_name)
# print("DONE")
%matplotlib qt

# data_sample = random.choices(data,k=3)
data_sample = data

show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

100%|██████████| 10/10 [00:01<00:00,  5.65it/s]


10


In [30]:
X,Y, data = mr.load_dataset("forces_only_f", filter_data = True, base_path=data_folder)


loading dataset:  forces_only_f ...done
raw X: (100, 953) 	Y: (100, 160)
filtered X: (100, 953) 	Y: (100, 160)


In [32]:
data_sample = []
c =0
for i,d in enumerate(data):
    if d["change_type"] == "force":    
        for o in d["obj_names"]:
            if o in d["text"]:
                break
        else:
            print("====================")
            print(d["text"])
            print(d["obj_names"])
            print(d["map_id"])
            c+=1
    

    # if "stretcher" in d["text"]:

    #     print("====================")
    #     # print(i)
    #     print(d["map_id"])
    #     # print(d["obj"])
    #     print(d["obj_names"])
    #     data_sample.append(d)

    # if "stretcher" in d["obj_names"]:
    #     print("+++++++++++++++++++")
    #     print(d["map_id"])
    #     print(d["obj_names"])
    #     data_sample.append(d)
print(c)
# print(sum([1 for d in data if d["map_id"]==0]))


0


In [19]:
# data_sample = random.choices(data,k=10)
show_data4D(data_sample,plot_output=False)#,image_loader=mr.image_loader)

In [54]:
from src.functions import *
import tensorflow as tf

embedding_indices = np.concatenate([feature_indices,obj_sim_indices, obj_poses_indices])

def prepare_x(x):
  objs = list_to_wp_seq(x[:,obj_poses_indices],d=4)
  trajs = list_to_wp_seq(x[:,traj_indices],d=4)
  return np.concatenate([objs,trajs],axis = 1)
ds = tf.data.Dataset.from_tensor_slices((prepare_x(X),list_to_wp_seq(Y,d=4),X[:,embedding_indices]))

2022-06-03 18:44:39.159195: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-03 18:44:39.159293: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-03 18:44:39.159391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-03 18:44:39.160092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 82 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 871.81GiB/s
2022-06-03 18:44:39.160112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-06-03 18:44:39.161444: I tensorflow/stream_executor/platform/de

In [58]:
def custom_shard_func(element):
  return 0

tf.data.experimental.save(
    ds, "banana_ds"
)


In [59]:
new_dataset = tf.data.experimental.load("banana_ds")

TypeError: load() missing 1 required positional argument: 'element_spec'